# Persistencia de dados

Durante o desenvolvimento de apps, nos deperamos com situações onde precisamos precisamos persisti os dados, nem sempre em um servidor remoto, pois nosso usuário pode utilizar a aplicação mesmo sem conexão de uma rede.

Por tanto permite o usa da aplicação offline é um habilidade que o desenvolvedor Mobile deve possuir!

Nesse capitulo iremos aprender a persistir dados localmente utilizando a **Biblioteca Room**, que oferece uma camada de abstração sobre o SQLite livrando o dev da construção de códigos extensos em SQL, salvo o SELECT!

Borá la!


## Implementando o Room no Projeto Android

- Criamos um projeto Android Studio Compose e o nomeamos de "Meus Contatos"
- Antes de começarmos vamos adicionar as dependências da biblioteca Room em nosso projeto, então abra o arquivo "build.gradle(module:app)" e adicione na sessão "dependencies" as seguintes linhas:

In [ ]:
//    Implementação do ROOM

        val room_version = "2.6.1"

    implementation("androidx.room:room-runtime:$room_version")
    annotationProcessor("androidx.room:room-compiler:$room_version")
    

Além disso precisaremos também inserir alguns plugins, neste mesmo arquivo build.gradle, na sessão de plugins e adicionar:

In [ ]:
plugins {
    alias(libs.plugins.androidApplication)
    alias(libs.plugins.jetbrainsKotlinAndroid)
    //Adicionar este \/
    id("kotlin-kapt")

}

Após adicionar o kapt nos plugins, vamos sincronizar com "Sync Now", com a sincronização concluída, podemos adicionar mais uma dependencia, a dependencia do kapt:

In [ ]:
    //Adicionando o kapt
    kapt("androidx.room:room-compiler:$room_version")

Podemos usar também o KSP, mas usaremos o kapt nessa atividade, mesmo sabendo que o KSP tem um desempenho melhor que o KAPT.

## Com o Room implementado.. seguimos

Lembrando que para trabalhar com ROOM, devemos usar classes, e cada classe possui um serie de parametros.

Por tanto vamos criar um pacote com os models (classes), no caso criaremos uma "Data Class" chamada "Produtos", dataClass pois essa classe só armazenara dados.

E criamos a classe data com seus atributos:

In [ ]:
package br.com.fiap.cadastrodeproduto.model

data class Produto(
    var id:Int = 0,
    var nome:String = "",
    var quantidade: Int = 0,
    var dataVencimento: String = "",
    var disponivel: Boolean = false
)

E para utiliza-lo é simples, como ja deixamos alguns valores default, basta chamar a classe e mudar o parametro que desejar:

In [ ]:
//Aqui a var produto que recebe Produto com o nome andrezao e os demais valor default
var prod = Produto(nome = "WheyProtein", disponivel = true)


## Agora precisamos apontar algumas infos da nossa classe

Para que o SQLite possa entende, precisamos definir algumas infos como:

- É um entity?
- Qual o nome da entity na tabela?
- Qual a chave primaria?
- A chave primaria é autogerada?
- Qual o nome da coluna e se ela possui um default value

Para apontar este basta seguir o exemplo abaixo:

In [ ]:
@Entity(tableName = "tbl_produtos")             //aqui passamos que será um entidade, com o nometabela de tbl_produtos

data class Produto(
    @PrimaryKey(autoGenerate = true)           //definimos que o primeiro atributo é chave primaria e também será autogerado
    var id:Long = 0,

    var nome:String = "",
    var quantidade: Int = 0,

    @ColumnInfo(name = "data_vencimento", defaultValue = "2024-01-01")          //Definimos que esta coluna tera o nome de "data_vencimento" e um valor padrão
    var dataVencimento: String = "",
    
    var disponivel: Boolean = false
)

## Criar uma interface para orientar o ROOM 

- Para isso criamos um pacote chamado "dao"

- Criamos um arquivo chamado "ProdutoDao" do tipo **interface** 

- Precisamos ter uma interface para cada produto! Estas interfaces dizem quais os comandos de persistencia este objeto fara

- Para que o ROOM saiba qual ele deve observar usaremos na interface o decorator **@Dao**

- Agora criamos a função que desejamos e a cima dela usamos um dos seguintes decorators:

 - **@Insert**, para inserir

 - **@Update**, para atualizar

 - **@Query**, que busca algo no banco porém este precisa ter o Select personalizado

 - **@Deleta**, que deleta algo no banco, passando o id ou o objeto em si

 Exemplo da aplicação:

In [ ]:
@Dao
interface ProdutoDao {

    //função salvar, que passaremos um produto do tipo Produto e retornara um Long que é o ID
    @Insert
    fun salvar(produto: Produto): Long

    //função que atualiza produto do tipo lista de produto e retorna a quantidade de itens atualizados
    @Update
    fun atualizar(produto: List<Produto>): Int

    //função que lista todos os produtos e retorna uma lista de produtos em ordem por nome
    @Query("SELECT * FROM tbl_produtos ORDER BY nome")
    fun listarTodosOsProdutos(): List<Produto>

    //função que busca por id
    @Query("SELECT * FROM tbl_produtos WHERE id = :id")
    fun buscarProdutoId(id:Long): Produto

    //Função que deleta um produto e diz quanto foram deletados
    @Delete
    fun excluir(produto:Produto): Int
}

## Apos criarmos o DAO, precisamos também criar o repository

..mas a aula parou nisso, vou buscar o conteúdo na materia do portal

## SQLite

SQLite, trabalha com o **ORM (Object-Relational Mapping)**, onde temos a classe e a entidade(tabela) que a representa.

Por tanto criamos a classe e o SQLite cria essa entidade e a tabela automaticamente

Um banco de dados leve(simples), onde podemos guardar 4x tipos de dados:

- Integer

- Real

- Strings

- Blob (video e imagens)

- Os tipos boolean, vamos armazenar em forma de números internos, ou seja 0(false) e 1(true)

- Data e hora, devemos armazenar em formato texto, como "YYYY-MM-DD HH:MM:SS"

- Outra opção para data e hora e pegar os milisegundos da data, seja em real ou interger, mas sinceramente esta opção é um pouco mais complexa do que a opção String com formatação padrão

Mais infos em https://www.sqlite.org/datatype3.html e recomendamos um breve leitura

### SQLite e anotação

Para indicar que uma classe é uma entidade, precisaremos apenas usar o anotation ou decorator (como preferir), adicionar o <code>@entity</code> em cima da classe desejada

..parei no video aos 31:34 da live trilha android 3